> Here, we use GA to adjust the structure of Logic Controller.

In [1]:
import numpy as np
from scipy.linalg import expm
import random

class move_gym():
    def __init__(self):
        self.scope = 2.0
        self.states = 6
        self.actions = 4
    def reset(self, s=[]):
        if np.array( s ).shape[0] == 0:
            self.obstacle_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.obstacle_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1

            self.move_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.move_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1  
            while ( abs(self.obstacle_x-self.move_x) < 1.05 ) & ( abs(self.obstacle_y-self.move_y) < 1.05 ):
                self.move_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
                self.move_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1      

            self.target_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.target_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            while (  ( abs(self.obstacle_x-self.target_x) < 1.05 ) & ( abs(self.obstacle_y-self.target_y) < 1.05 )  ) |\
             (  ( abs(self.move_x-self.target_x) < 1.05 ) & ( abs(self.move_y-self.target_y) < 1.05 )  ):
                self.target_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
                self.target_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1        
        else:
            self.obstacle_x=s[0]
            self.obstacle_y=s[1]
            self.target_x=s[2]
            self.target_y=s[3]
            self.move_x=s[4]
            self.move_y=s[5]

        state=np.array([ self.obstacle_x, self.obstacle_y, self.target_x, self.target_y, self.move_x, self.move_y ])
        return state #, self.state2img(state)
    def step(self, action):
        velocity = 0.2
        if action==0: # up down right left
            self.move_y+=velocity
        if action==1:
            self.move_y-=velocity
        if action==2:
            self.move_x+=velocity
        if action==3:
            self.move_x-=velocity

        if self.move_x > (self.scope+1.0):
            self.move_x-=velocity
        if self.move_x < (-self.scope-1.0):
            self.move_x+=velocity
        if self.move_y > (self.scope+1.0):
            self.move_y-=velocity
        if self.move_y < (-self.scope-1.0):
            self.move_y+=velocity
        
        reward = -0.1
        done = False
        info = "^_^"
        if (  ( abs(self.obstacle_x-self.move_x) < 1.05 ) & ( abs(self.obstacle_y-self.move_y) < 1.05 )  ): 
            reward = -1.0 
            done = True
            info = "collision"

        elif (  ( abs(self.target_x-self.move_x) < 1.05 ) & ( abs(self.target_y-self.move_y) < 1.05 )  ):
            reward = 1.0
            done = True
            info = "reach"

        state=np.array([ self.obstacle_x, self.obstacle_y, self.target_x, self.target_y, self.move_x, self.move_y ])

        return state, reward,done,info
    def state2img(self, state):
        img = np.zeros([84,84,3], dtype=np.uint8)
        ( img[ int(42-(self.obstacle_y+0.5)*10):int(42-(self.obstacle_y-0.5)*10), int((self.obstacle_x-0.5)*10+42):int((self.obstacle_x+0.5)*10+42),2 ] ).fill(255)
        ( img[ int(42-(self.target_y+0.5)*10):int(42-(self.target_y-0.5)*10), int((self.target_x-0.5)*10+42):int((self.target_x+0.5)*10+42),1 ] ).fill(255)
        ( img[ int(42-(self.move_y+0.5)*10):int(42-(self.move_y-0.5)*10), int((self.move_x-0.5)*10+42):int((self.move_x+0.5)*10+42),0 ] ).fill(255)
        return img

In [2]:
import tensorflow as tf
import numpy as np
from numpy import *
import random
from collections import deque

from tqdm import trange

# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch

class DQN():
    # DQN Agent
    def __init__(self, env):
        # init experience replay
        self.replay_buffer = deque()
        # init. some parameters
        self.epsilon = INITIAL_EPSILON
        
        # self.state_dim = env.observation_space.shape[0]
        self.state_dim = env.states

        # self.action_dim = env.action_space.n
        self.action_dim = env.actions

        self.create_Q_network()
        self.create_training_method()

        # Init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())

    def create_Q_network(self):
        
#         # # a: create new network weights
#         print "create new network weights ..."
#         num_hid = 200
#         self.W1 = self.weight_variable([self.state_dim,num_hid])
#         self.b1 = self.bias_variable([num_hid])
#         self.W2 = self.weight_variable([num_hid,num_hid])
#         self.b2 = self.bias_variable([num_hid])
#         self.W3 = self.weight_variable([num_hid,self.action_dim])
#         self.b3= self.bias_variable([self.action_dim])

        # # b: restore old network weights
        print "restore old network weights ..."
        self.W1, self.b1, self.W2, self.b2, self.W3, self.b3 = self.restore_wb( "train-3500/" )


        # input layer
        self.state_input = tf.placeholder("float",[None,self.state_dim])
        # hidden layers
        h_layer = tf.nn.relu(tf.matmul(self.state_input,self.W1) + self.b1)
        h_layer02 = tf.nn.relu(tf.matmul(h_layer,self.W2) + self.b2)
        # Q Value layer
        self.Q_value = tf.matmul(h_layer02,self.W3) + self.b3

    def create_training_method(self):
        self.action_input = tf.placeholder("float",[None,self.action_dim]) # one hot presentation
        self.y_input = tf.placeholder("float",[None])
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value,self.action_input),reduction_indices = 1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)

    def perceive(self,state,action,reward,next_state,done):
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))
        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()

    def train_Q_network(self):
        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]

        # Step 2: calculate y
        y_batch = []
        Q_value_batch = self.Q_value.eval(feed_dict={self.state_input:next_state_batch})
        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if done:
                y_batch.append(reward_batch[i])
            else :
                y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))

        self.optimizer.run(feed_dict={
          self.y_input:y_batch,
          self.action_input:action_batch,
          self.state_input:state_batch
          })

    def egreedy_action(self,state):
        Q_value = self.Q_value.eval(feed_dict = {
          self.state_input:[state]
          })[0]
        if random.random() <= 0.5:
            return random.randint(0,self.action_dim - 1)
        else:
            return np.argmax(Q_value)

    def obtain_wb(self):
        W1=self.session.run(self.W1)
        b1=self.session.run(self.b1)
        W2=self.session.run(self.W2)
        b2=self.session.run(self.b2)
        W3=self.session.run(self.W3)
        b3=self.session.run(self.b3)
        return W1, b1, W2, b2, W3, b3

    def restore_wb(self, directory):
        W1 = np.load( directory+"W1.npy" )
        W2 = np.load( directory+"W2.npy" )
        W3 = np.load( directory+"W3.npy" )

        b1 = np.load( directory+"b1.npy" )
        b2 = np.load( directory+"b2.npy" )
        b3 = np.load( directory+"b3.npy" )        
        return tf.Variable(W1), tf.Variable(b1), tf.Variable(W2), tf.Variable(b2), tf.Variable(W3), tf.Variable(b3)
    def use_nn(self,state):
        return self.Q_value.eval(feed_dict = {
          self.state_input:[state]
          })

    def action(self,state):
        return np.argmax(self.Q_value.eval(feed_dict = {
          self.state_input:[state]
          })[0])

    def weight_variable(self,shape):
        initial = tf.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self,shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)
    
    
# Hyper Parameters
STEP = 50 # Step limitation in an episode
TEST = 10 # The number of experiment for test

# initialize env and agent
env = move_gym()
agent = DQN(env)

all_episode = 0
all_reward = []

restore old network weights ...


In [3]:
def discrete_it(state, discrete):
    
    dis_s = []
    for i in xrange( len(state) ):
        num = state[i]
        over = num - discrete
        for j in xrange( len(over) ):
            if over[j]<0:
                over[j] = 100 # a certain big number
        dis_s.append( np.argmin( over ) )
    return dis_s

In [4]:
def state2relative(state):
    top_t = 100.0
    bottom_t = 100.0
    right_t = 100.0
    left_t = 100.0

    top_o = 100.0
    bottom_o = 100.0
    right_o = 100.0
    left_o = 100.0        

    obstacle_x=state[0]
    obstacle_y=state[1]
    target_x=state[2]
    target_y=state[3]
    move_x=state[4]
    move_y=state[5]

    if target_y >= move_y:
        top_t = target_y - move_y
    if target_y < move_y:
        bottom_t = move_y - target_y
    if target_x >= move_x:
        right_t = target_x - move_x
    if target_x < move_x:
        left_t = move_x - target_x

    if obstacle_y >= move_y:
        top_o = obstacle_y - move_y
    if obstacle_y < move_y:
        bottom_o = move_y - obstacle_y
    if obstacle_x >= move_x:
        right_o = obstacle_x - move_x
    if obstacle_x < move_x:
        left_o = move_x - obstacle_x

    return np.array([ top_t, bottom_t, right_t, left_t, top_o, bottom_o, right_o, left_o ])

In [5]:
def logic_create(discrete):
    logic_q = np.zeros( [6,6,6,6, 6,6,6,6, 5] )
    
    for it_tms in xrange(10):
        info ="^_^"
        good_state = False
        for i in xrange(3000):
            if (info=="reach") & (good_state == False):
                state = env.reset(A)
#                 C.append(A)
                good_state = True
            else:
                state = env.reset()
                good_state = False
            A = state
            for j in xrange(STEP):
                relative_state=state2relative(state)
                # discrete it:
                relative_state = discrete_it( relative_state, discrete )

                action = agent.action(state)
                if good_state:
                    if max(logic_q[relative_state[0]][relative_state[1]][relative_state[2]][relative_state[3]]\
                        [relative_state[4]][relative_state[5]][relative_state[6]][relative_state[7]])<1000:

                        logic_q[relative_state[0]][relative_state[1]][relative_state[2]][relative_state[3]]\
                        [relative_state[4]][relative_state[5]][relative_state[6]][relative_state[7]][action+1] +=1

                state, reward,done,info = env.step(action)
                if done:
                    break

        ###############################
        if it_tms==9:
            B = 0
            B11= 0

            for i in xrange(1000):
                state = env.reset()
                A = state
                for j in xrange(STEP):
                    relative_state=state2relative(state)
                    # discrete it:
                    relative_state = discrete_it( relative_state, discrete )

                    action = np.argmax( logic_q[relative_state[0]][relative_state[1]][relative_state[2]][relative_state[3]]\
                    [relative_state[4]][relative_state[5]][relative_state[6]][relative_state[7]] )-1
                    state, reward,done,info = env.step(action)
                    if done:
                        if info=="collision":
#                             B1.append(A)
#                             B11.append(A)
                                B11 += 1
                        break
                if done == 0:
#                     B.append(A)
                        B += 1

        ###############################
    
#             print "len(B): ", len(B)
#             print "len(B11): ", len(B11)
#             print "len(B1): ", len(B1)
#             np.save("logic_q-only", logic_q)
#             np.save("C", C)

    return B, B11 # len(B), len(B11)

In [64]:
from pyevolve import G1DList

from pyevolve import GSimpleGA


# This function is the evaluation function, we want

# to give high score to more zero'ed chromosomes

ink = 0

def eval_func(chromosome):
    global ink
    A=[]
    for value in chromosome:
        A.append( value )
    discrete = np.array( [0.,         1.0875,     1.48,       2.2909375,  3.60625,    4.475] )

#     B = [0.8, 1.2, 1.84, 2.95, 3.7, 4.5]
    B = [0.5, 1.25, 1.9, 3.0, 4.0, 5.0]
    
    for i in range(5):
        discrete[i+1] = B[i]+A[i]*(B[i+1]-B[i])/64.0
    
    n_no_done, n_collision = logic_create(discrete)
    
    print ">>: ", ink, n_no_done, n_collision, discrete
    ink += 1

    return 1000 - n_no_done - n_collision
    
#    score = 0.0

#    # iterate over the chromosome elements (items)

#    for value in chromosome:

#       if value==0:

#          score += 1.0

#    return score+1000


# Genome instance

genome = G1DList.G1DList(5)

genome.setParams(rangemin=0, rangemax=63)

# The evaluator function (objective function)

genome.evaluator.set(eval_func)

ga = GSimpleGA.GSimpleGA(genome)

ga.setGenerations(9)
ga.setPopulationSize(50)

# Do the evolution, with stats dump

# frequency of 10 generations

ga.evolve(freq_stats=10)


# Best individual

print ga.bestIndividual()

>>:  0 173 14 [ 0.          1.19140625  1.34140625  2.828125    3.671875    4.78125   ]
>>:  1 58 6 [ 0.          1.0859375   1.68671875  2.5015625   3.0625      4.        ]
>>:  2 65 147 [ 0.          0.86328125  1.3515625   2.8109375   3.328125    4.953125  ]
>>:  3 55 125 [ 0.          0.5234375   1.58515625  2.828125    3.          4.953125  ]
>>:  4 62 142 [ 0.         0.8984375  1.33125    2.484375   3.125      4.34375  ]
>>:  5 47 161 [ 0.          0.7578125   1.50390625  2.6390625   3.03125     4.65625   ]
>>:  6 277 96 [ 0.          1.203125    1.52421875  2.7765625   3.40625     4.25      ]
>>:  7 172 32 [ 0.          1.16796875  1.5953125   2.003125    3.765625    4.53125   ]
>>:  8 266 103 [ 0.          1.21484375  1.88984375  2.896875    3.8125      4.96875   ]
>>:  9 55 57 [ 0.         0.96875    1.371875   2.3296875  3.109375   4.484375 ]
>>:  10 46 164 [ 0.          0.60546875  1.68671875  2.553125    3.5625      4.3125    ]
>>:  11 45 147 [ 0.          0.7109375   1.46

>>:  0 253 80 [ 0.    1.25  1.9   2.55  2.9   4.1 ]
>>:  1 305 143 [ 0.    1.5   2.8   3.15  4.    5.35]
>>:  2 16 195 [ 0.    0.5   2.    2.8   4.15  4.75]
>>:  3 306 156 [ 0.    1.5   2.25  3.7   4.25  5.8 ]
>>:  4 94 111 [ 0.    0.9   1.4   2.    2.75  3.45]
>>:  5 71 51 [ 0.    1.    1.55  2.25  2.25  3.2 ]
>>:  6 285 115 [ 0.    1.3   2.3   3.05  3.7   4.9 ]
>>:  7 56 28 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  8 292 147 [ 0.    1.35  1.6   1.75  2.85  3.6 ]
>>:  9 157 111 [ 0.    1.2   2.45  3.5   4.9   5.2 ]
>>:  10 69 104 [ 0.    0.3   1.3   2.6   3.3   4.15]
>>:  11 261 160 [ 0.    1.35  2.65  3.75  4.55  5.25]
>>:  12 52 152 [ 0.    0.4   1.45  2.45  3.55  4.55]
>>:  13 69 151 [ 0.    0.8   1.4   1.75  2.3   3.55]
>>:  14 287 105 [ 0.    1.3   2.6   3.35  3.45  4.5 ]
>>:  15 57 71 [ 0.    1.05  1.15  2.35  2.45  3.95]
>>:  16 89 174 [ 0.    0.3   0.35  0.8   2.25  3.3 ]
>>:  17 413 59 [ 0.    0.15  0.75  1.2   1.45  1.7 ]
>>:  18 455 0 [ 0.    0.45  1.05  1.3   1.45  1.65]
>>:  19 207 130 [ 0.    0.05  1.35  2.45  2.85  3.5 ]
>>:  20 19 171 [ 0.    0.6   2.1   3.55  3.55  5.05]
>>:  21 66 139 [ 0.    0.05  0.9   2.4   3.    3.45]
>>:  22 46 156 [ 0.    0.3   1.7   2.4   3.5   4.65]
>>:  23 41 152 [ 0.    0.05  0.7   1.6   3.05  4.25]
>>:  24 281 182 [ 0.    1.55  2.    2.95  2.95  3.5 ]
>>:  25 311 86 [ 0.    1.3   1.85  2.3   3.55  3.85]
>>:  26 308 172 [ 0.    1.45  2.    3.05  4.35  5.3 ]
>>:  27 61 165 [ 0.    0.5   0.75  1.45  2.55  3.8 ]
>>:  28 119 186 [ 0.    0.6   0.8   1.95  2.45  3.2 ]
>>:  29 14 188 [ 0.    0.6   2.1   2.7   4.15  4.75]
>>:  30 118 128 [ 0.    0.2   1.5   1.75  2.85  3.75]
>>:  31 114 75 [ 0.    0.5   1.15  1.75  3.2   3.6 ]
>>:  32 281 164 [ 0.    1.55  2.7   2.9   3.65  5.05]
>>:  33 74 137 [ 0.    0.25  1.75  3.2   3.35  4.7 ]
>>:  34 48 173 [ 0.    0.25  0.6   2.1   3.2   4.1 ]
>>:  35 180 97 [ 0.    0.05  0.15  0.65  1.2   2.35]
>>:  36 157 100 [ 0.    0.55  1.2   1.85  2.3   2.75]
>>:  37 55 151 [ 0.    0.5   1.75  2.7   3.6   4.45]
>>:  38 37 175 [ 0.    0.85  2.3   2.9   4.25  5.25]
>>:  39 50 178 [ 0.    0.5   2.05  2.35  3.65  5.2 ]
>>:  40 26 195 [ 0.    0.    0.85  2.1   3.15  4.6 ]
>>:  41 267 95 [ 0.    1.3   2.2   3.45  4.05  4.95]
>>:  42 99 180 [ 0.    0.4   0.6   2.1   2.35  3.6 ]
>>:  43 77 77 [ 0.    0.25  0.95  1.9   3.05  3.7 ]
>>:  44 86 185 [ 0.    0.3   0.35  1.9   2.95  4.  ]
>>:  45 110 142 [ 0.   0.2  0.2  1.6  2.4  2.7]
>>:  46 276 103 [ 0.    1.25  1.8   2.1   3.45  4.75]
>>:  47 28 113 [ 0.    0.9   1.95  3.45  4.75  5.7 ]
>>:  48 187 142 [ 0.    1.2   1.85  3.25  3.85  4.15]
>>:  49 64 178 [ 0.    0.65  0.75  1.7   3.2   4.25]
Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [863.47(916.00)/573.36(520.00)/719.56(719.56)]
>>:  50 73 27 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  51 53 19 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  52 49 30 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  53 67 27 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  54 59 19 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  55 69 135 [ 0.    0.4   1.35  2.45  3.2   4.05]
>>:  56 54 25 [ 0.    1.1   1.95  2.35  3.1   4.05]
>>:  57 62 19 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  58 63 25 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  59 65 30 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  60 79 21 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  61 58 26 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  62 68 13 [ 0.    1.1   1.95  2.35  3.1   4.05]
>>:  63 57 31 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  64 61 21 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  65 61 31 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  66 73 28 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  67 57 42 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  68 56 40 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  69 58 22 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  70 47 37 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  71 42 34 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  72 49 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  73 59 27 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  74 51 31 [ 0.    1.1   2.05  2.45  3.3   4.05]
>>:  75 55 32 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  76 50 29 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  77 45 22 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  78 68 40 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  79 41 85 [ 0.    0.95  2.05  2.45  3.2   4.05]
>>:  80 57 38 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  81 62 31 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  82 65 21 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  83 63 23 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  84 46 24 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  85 54 25 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  86 71 35 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  87 60 17 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  88 76 28 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  89 56 36 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  90 58 37 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  91 46 27 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  92 56 20 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  93 49 20 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  94 46 40 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  95 59 17 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  96 62 35 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  97 60 29 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  98 43 26 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  99 59 25 [ 0.    1.1   2.05  2.45  3.2   4.05]
>>:  100 64 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  101 53 102 [ 0.    0.85  1.25  2.2   2.95  4.05]
>>:  102 56 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  103 56 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  104 61 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  105 66 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  106 78 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  107 47 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  108 44 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  109 69 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  110 71 5 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  111 66 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  112 67 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  113 69 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  114 69 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  115 72 0 [ 0.    1.1   1.5   2.25  3.2   4.05]
>>:  116 60 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  117 75 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  118 72 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  119 58 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  120 69 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  121 58 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  122 61 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  123 53 3 [ 0.    1.1   1.5   2.45  3.3   4.05]
>>:  124 53 142 [ 0.    0.4   1.5   2.45  3.2   4.05]
>>:  125 61 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  126 62 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  127 63 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  128 57 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  129 63 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  130 46 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  131 60 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  132 53 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  133 55 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  134 69 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  135 61 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  136 61 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  137 51 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  138 60 5 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  139 64 1 [ 0.    1.1   1.5   2.25  3.1   4.05]
>>:  140 70 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  141 56 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  142 57 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  143 55 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  144 66 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  145 65 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  146 66 5 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  147 68 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  148 60 106 [ 0.    0.85  1.25  2.2   2.95  4.05]
>>:  149 60 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  150 56 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  151 71 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  152 67 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  153 62 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  154 56 26 [ 0.    1.1   1.95  2.9   3.65  4.05]
>>:  155 72 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  156 57 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  157 45 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  158 62 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  159 63 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  160 63 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  161 56 7 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  162 64 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  163 71 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  164 59 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  165 68 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  166 61 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  167 56 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  168 53 2 [ 0.    1.1   1.5   2.35  3.1   4.05]
>>:  169 64 1 [ 0.    1.1   1.5   2.25  3.2   4.05]
>>:  170 63 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  171 64 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  172 49 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  173 69 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  174 63 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  175 42 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  176 58 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  177 66 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  178 69 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  179 63 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  180 61 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  181 61 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  182 64 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  183 64 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  184 55 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  185 60 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  186 56 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  187 72 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  188 61 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  189 66 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  190 72 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  191 62 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  192 73 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  193 62 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  194 60 0 [ 0.    1.1   1.5   2.35  3.1   4.05]
>>:  195 49 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  196 65 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  197 64 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  198 73 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  199 71 5 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  200 78 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  201 64 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  202 59 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  203 50 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  204 67 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  205 61 5 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  206 64 4 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  207 77 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  208 49 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  209 77 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  210 52 3 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  211 63 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  212 62 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  213 55 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  214 66 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  215 71 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  216 54 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  217 65 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  218 63 6 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  219 56 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  220 59 0 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  221 42 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  222 58 2 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  223 60 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  224 72 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  225 74 0 [ 0.    1.1   1.5   2.45  3.2   4.05]

>>:  0 188 52 [ 0.    1.17  1.54  2.33  3.13  4.06]
>>:  1 58 86 [ 0.    0.95  1.39  2.41  3.32  4.18]
>>:  2 49 75 [ 0.    0.97  1.6   2.29  3.15  3.94]
>>:  3 59 82 [ 0.    0.99  1.62  2.33  3.04  4.17]
>>:  4 303 104 [ 0.    1.23  1.34  2.46  3.29  3.89]
>>:  5 63 3 [ 0.    1.1   1.42  2.59  3.11  3.96]
>>:  6 63 0 [ 0.    1.1   1.39  2.56  3.04  3.98]
>>:  7 69 81 [ 0.    0.95  1.46  2.41  3.3   3.92]
>>:  8 203 46 [ 0.    1.16  1.64  2.47  3.12  4.03]
>>:  9 66 6 [ 0.    1.1   1.63  2.38  3.24  4.08]
>>:  10 189 21 [ 0.    1.18  1.37  2.56  3.13  4.11]
>>:  11 159 29 [ 0.    1.15  1.34  2.38  3.09  4.04]
>>:  12 61 1 [ 0.    1.05  1.41  2.52  3.05  4.08]
>>:  13 52 52 [ 0.    1.    1.42  2.59  3.09  4.07]
>>:  14 67 0 [ 0.    1.03  1.36  2.32  3.15  4.15]
>>:  15 178 16 [ 0.    1.16  1.39  2.29  3.34  3.96]
>>:  16 54 81 [ 0.    0.97  1.42  2.47  3.22  4.02]
>>:  17 55 2 [ 0.    1.03  1.41  2.4   3.32  3.99]
>>:  18 53 81 [ 0.    0.99  1.48  2.42  3.25  3.97]
>>:  19 253 91 [ 0.    1.24  1.39  2.3   3.17  4.13]
>>:  20 54 77 [ 0.    0.95  1.61  2.48  3.22  4.13]
>>:  21 51 4 [ 0.    1.1   1.57  2.47  3.14  3.89]
>>:  22 169 16 [ 0.    1.19  1.36  2.32  3.1   4.19]
>>:  23 58 0 [ 0.    1.03  1.4   2.33  3.34  4.  ]
>>:  24 273 85 [ 0.    1.22  1.5   2.57  3.11  4.06]
>>:  25 193 19 [ 0.    1.17  1.4   2.45  3.16  4.04]
>>:  26 159 26 [ 0.    1.14  1.42  2.37  3.17  3.98]
>>:  27 191 26 [ 0.    1.12  1.35  2.38  3.23  4.  ]
>>:  28 51 0 [ 0.    1.02  1.58  2.36  3.21  4.  ]
>>:  29 189 10 [ 0.    1.12  1.37  2.51  3.11  3.97]
>>:  30 205 13 [ 0.    1.16  1.38  2.33  3.27  4.04]
>>:  31 55 73 [ 0.    0.96  1.49  2.5   3.1   4.18]
>>:  32 59 0 [ 0.    1.08  1.35  2.52  3.27  4.16]
>>:  33 61 75 [ 0.    0.98  1.4   2.56  3.07  4.07]
>>:  34 196 22 [ 0.    1.12  1.4   2.45  3.1   4.1 ]
>>:  35 171 18 [ 0.    1.18  1.45  2.49  3.11  4.1 ]
>>:  36 41 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  37 177 30 [ 0.    1.15  1.42  2.4   3.25  3.98]
>>:  38 69 0 [ 0.    1.03  1.41  2.41  3.22  4.17]
>>:  39 46 1 [ 0.    1.09  1.56  2.36  3.13  4.13]
>>:  40 51 1 [ 0.    1.05  1.44  2.43  3.28  3.97]
>>:  41 173 27 [ 0.    1.12  1.53  2.54  3.08  4.1 ]
>>:  42 266 109 [ 0.    1.22  1.41  2.37  3.34  3.92]
>>:  43 186 40 [ 0.    1.13  1.44  2.3   3.28  4.19]
>>:  44 176 49 [ 0.    1.19  1.64  2.6   3.06  3.93]
>>:  45 64 11 [ 0.    1.1   1.53  2.51  3.21  4.12]
>>:  46 52 0 [ 0.    1.01  1.57  2.33  3.05  3.94]
>>:  47 284 100 [ 0.    1.23  1.44  2.56  3.19  3.95]
>>:  48 51 80 [ 0.    0.98  1.5   2.37  3.24  4.13]
>>:  49 198 15 [ 0.    1.14  1.37  2.35  3.3   3.95]
Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [1006.87(959.00)/494.79(593.00)/839.06(839.06)]
>>:  50 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  51 71 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  52 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  53 45 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  54 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  55 63 0 [ 0.    1.09  1.34  2.4   3.35  4.06]
>>:  56 54 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  57 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  58 57 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  59 62 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  60 60 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  61 54 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  62 56 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  63 75 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  64 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  65 51 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  66 51 0 [ 0.    1.09  1.34  2.46  3.35  4.  ]
>>:  67 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  68 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  69 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  70 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  71 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  72 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  73 56 0 [ 0.    1.05  1.34  2.6   3.19  4.06]
>>:  74 70 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  75 55 3 [ 0.    1.05  1.65  2.44  3.04  4.06]
>>:  76 43 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  77 82 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  78 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  79 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  80 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  81 70 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  82 48 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  83 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  84 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  85 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  86 57 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  87 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  88 50 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  89 66 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  90 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  91 56 1 [ 0.    1.05  1.34  2.44  3.21  4.2 ]
>>:  92 56 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  93 70 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  94 71 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  95 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  96 63 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  97 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  98 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  99 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  100 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  101 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  102 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  103 60 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  104 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  105 63 3 [ 0.    1.09  1.34  2.4   3.35  4.06]
>>:  106 48 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  107 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  108 59 0 [ 0.    1.05  1.51  2.44  3.35  3.89]
>>:  109 49 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  110 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  111 47 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  112 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  113 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  114 77 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  115 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  116 64 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  117 56 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  118 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  119 60 1 [ 0.    1.05  1.49  2.29  3.35  4.06]
>>:  120 50 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  121 70 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  122 66 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  123 278 107 [ 0.    1.25  1.34  2.44  3.15  4.06]
>>:  124 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  125 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  126 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  127 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  128 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  129 50 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  130 45 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  131 51 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  132 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  133 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  134 50 75 [ 0.    0.94  1.45  2.44  3.35  4.06]
>>:  135 47 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  136 165 17 [ 0.    1.11  1.34  2.44  3.35  4.  ]
>>:  137 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  138 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  139 42 5 [ 0.    1.05  1.65  2.44  3.04  4.06]
>>:  140 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  141 64 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  142 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  143 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  144 64 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  145 52 67 [ 0.    0.94  1.45  2.44  3.35  4.06]
>>:  146 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  147 56 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  148 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  149 62 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  150 62 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  151 51 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  152 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  153 48 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  154 72 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  155 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  156 46 7 [ 0.    1.05  1.65  2.44  3.04  4.06]
>>:  157 52 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  158 53 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  159 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  160 50 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  161 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  162 71 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  163 56 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  164 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  165 49 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  166 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  167 68 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  168 42 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  169 59 0 [ 0.    1.05  1.34  2.6   3.19  4.06]
>>:  170 57 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  171 62 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  172 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  173 45 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  174 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  175 61 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  176 62 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  177 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  178 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  179 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  180 46 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  181 60 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  182 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  183 70 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  184 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  185 61 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  186 64 3 [ 0.    1.05  1.51  2.44  3.35  3.89]
>>:  187 43 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  188 68 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  189 54 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  190 69 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  191 58 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  192 64 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  193 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  194 59 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  195 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  196 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  197 57 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  198 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  199 46 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  200 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  201 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  202 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  203 56 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  204 49 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  205 48 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  206 77 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  207 64 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  208 52 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  209 45 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  210 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  211 55 3 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  212 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  213 66 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  214 56 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  215 50 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  216 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  217 43 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  218 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  219 68 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  220 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  221 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  222 61 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  223 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  224 51 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  225 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  226 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  227 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  228 62 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  229 46 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  230 42 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  231 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  232 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  233 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  234 54 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  235 51 1 [ 0.    1.05  1.34  2.6   3.19  4.06]
>>:  236 49 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  237 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  238 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  239 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  240 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  241 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  242 67 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  243 72 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  244 51 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  245 66 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  246 69 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  247 59 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  248 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  249 64 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  250 44 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  251 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  252 52 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  253 59 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  254 43 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  255 50 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  256 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  257 57 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  258 47 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  259 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  260 71 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  261 69 6 [ 0.    1.05  1.51  2.44  3.35  3.89]
>>:  262 70 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  263 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  264 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  265 41 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  266 72 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  267 52 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  268 70 2 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  269 65 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  270 60 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  271 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  272 58 0 [ 0.    1.05  1.34  2.6   3.19  4.06]
>>:  273 47 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  274 56 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  275 48 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  276 48 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  277 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  278 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  279 53 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  280 63 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  281 61 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  282 39 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  283 54 0 [ 0.    1.05  1.34  2.46  3.35  4.04]
>>:  284 58 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  285 65 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  286 48 5 [ 0.    1.05  1.65  2.44  3.04  4.06]
>>:  287 54 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  288 52 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  289 71 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  290 51 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  291 62 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  292 50 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  293 48 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  294 50 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  295 49 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  296 83 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  297 47 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  298 43 1 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  299 55 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  300 162 11 [ 0.    1.11  1.34  2.44  3.35  4.  ]
>>:  301 51 1 [ 0.    1.05  1.34  2.44  3.35  4.06]

In [ ]:
>>:  0 60 159 [ 0.         0.8        1.52       1.84       3.6765625  3.725    ]
>>:  1 39 88 [ 0.         0.925      1.78       2.2909375  3.3953125  4.375    ]
>>:  2 57 152 [ 0.        0.8375    1.36      1.84      3.465625  3.925   ]
>>:  3 65 50 [ 0.         1.         1.66       2.1175     3.2078125  3.725    ]
>>:  4 52 112 [ 0.        0.8125    1.2       2.048125  3.559375  4.325   ]
>>:  5 59 70 [ 0.         0.9625     1.38       2.2215625  3.371875   4.275    ]
>>:  6 159 14 [ 0.         1.1375     1.38       2.81125    3.5359375  4.4      ]
>>:  7 43 169 [ 0.         0.85       1.74       2.8459375  3.2546875  4.25     ]
>>:  8 42 0 [ 0.         1.05       1.26       2.1521875  2.996875   4.025    ]
>>:  9 57 88 [ 0.         0.975      1.8        2.4296875  3.1375     4.15     ]
>>:  10 58 56 [ 0.         1.         1.32       2.53375    2.9734375  4.075    ]
>>:  11 59 75 [ 0.         0.975      1.22       2.048125   3.1609375  3.875    ]
>>:  12 45 78 [ 0.         0.9125     1.46       1.84       3.5359375  4.125    ]
>>:  13 81 148 [ 0.         0.8        1.32       2.4296875  3.184375   3.7      ]
>>:  14 171 23 [ 0.         1.15       1.44       2.395      3.4421875  4.225    ]
>>:  15 56 82 [ 0.         0.9125     1.2        2.9153125  3.2546875  4.325    ]
>>:  16 46 142 [ 0.         0.8        1.2        2.0828125  3.465625   3.925    ]
>>:  17 163 17 [ 0.         1.15       1.34       2.0828125  3.0203125  4.35     ]
>>:  18 48 113 [ 0.        0.9       1.6       1.97875   2.996875  4.075   ]
>>:  19 188 38 [ 0.         1.125      1.52       2.1521875  3.559375   3.8      ]
>>:  20 186 28 [ 0.         1.175      1.56       2.7765625  3.60625    4.35     ]
>>:  21 57 74 [ 0.        0.975     1.46      2.741875  3.41875   3.8     ]
>>:  22 71 84 [ 0.         0.9875     1.6        2.395      3.5828125  3.7      ]
>>:  23 176 25 [ 0.         1.125      1.32       2.464375   3.6765625  3.775    ]
>>:  24 52 75 [ 0.        0.9125    1.46      1.909375  3.60625   4.2     ]
>>:  25 63 147 [ 0.        0.8125    1.4       2.186875  3.5125    4.      ]
>>:  26 166 75 [ 0.         1.125      1.82       2.048125   3.4890625  4.3      ]
>>:  27 47 117 [ 0.         0.8125     1.26       2.2909375  3.465625   3.875    ]
>>:  28 39 79 [ 0.        0.95      1.22      2.395     3.653125  3.9     ]
>>:  29 44 163 [ 0.         0.85       1.72       2.603125   3.3953125  4.325    ]
>>:  30 41 6 [ 0.        1.025     1.82      2.464375  3.41875   4.125   ]
>>:  31 60 165 [ 0.        0.8375    1.56      2.6725    3.371875  3.9     ]
>>:  32 170 43 [ 0.         1.1625     1.7        2.2909375  3.653125   3.775    ]
>>:  33 55 42 [ 0.         1.         1.78       1.8746875  3.4421875  4.15     ]
>>:  34 58 152 [ 0.        0.8875    1.42      2.25625   2.996875  3.825   ]
>>:  35 63 150 [ 0.      0.8625  1.42    2.1175  3.325   4.15  ]
>>:  36 40 140 [ 0.        0.8       1.32      2.880625  3.559375  4.35    ]
>>:  37 67 91 [ 0.         0.8375     1.28       2.0134375  3.2546875  3.825    ]
>>:  38 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  39 167 89 [ 0.         1.1125     1.2        2.4990625  3.4421875  4.025    ]
>>:  40 168 33 [ 0.         1.125      1.6        2.7765625  3.4421875  4.125    ]
>>:  41 40 122 [ 0.         0.825      1.26       1.84       3.6296875  4.4      ]
>>:  42 48 137 [ 0.         0.825      1.32       2.5684375  3.6765625  3.95     ]
>>:  43 172 76 [ 0.         1.125      1.8        2.0134375  3.23125    3.85     ]
>>:  44 52 89 [ 0.        0.9125    1.56      2.880625  3.559375  3.9     ]
>>:  45 55 5 [ 0.        1.0875    1.68      2.603125  3.1375    4.1     ]
>>:  46 53 87 [ 0.         0.925      1.54       2.464375   3.6296875  4.275    ]
>>:  47 48 82 [ 0.        0.9125    1.38      2.048125  3.559375  4.125   ]
>>:  48 79 168 [ 0.        0.8       1.46      2.880625  3.23125   3.775   ]
>>:  49 50 75 [ 0.      0.9875  1.42    2.6725  3.1375  4.025 ]
Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [1001.64(963.00)/716.68(744.00)/834.70(834.70)]
>>:  50 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  51 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  52 40 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  53 33 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  54 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  55 36 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  56 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  57 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  58 41 80 [ 0.         0.975      1.66       2.2909375  3.60625    4.475    ]
>>:  59 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  60 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  61 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  62 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  63 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  64 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  65 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  66 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  67 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  68 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  69 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  70 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  71 50 3 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  72 68 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  73 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  74 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  75 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  76 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  77 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  78 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  79 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  80 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  81 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  82 35 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  83 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  84 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  85 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  86 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  87 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  88 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  89 56 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  90 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  91 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  92 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  93 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  94 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  95 67 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  96 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  97 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  98 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  99 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  100 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  101 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  102 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  103 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  104 34 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  105 27 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  106 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  107 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  108 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  109 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  110 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  111 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  112 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  113 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  114 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  115 63 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  116 58 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  117 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  118 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  119 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  120 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  121 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  122 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  123 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  124 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  125 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  126 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  127 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  128 46 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  129 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  130 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  131 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  132 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  133 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  134 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  135 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  136 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  137 64 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  138 61 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  139 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  140 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  141 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  142 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  143 47 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  144 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  145 66 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  146 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  147 34 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  148 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  149 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  150 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  151 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  152 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  153 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  154 41 0 [ 0.         1.0875     1.48       2.2909375  3.6765625  4.4      ]
>>:  155 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  156 49 78 [ 0.         0.9625     1.48       2.6378125  3.60625    4.475    ]
>>:  157 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  158 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  159 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  160 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  161 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  162 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  163 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  164 68 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  165 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  166 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  167 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  168 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  169 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  170 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  171 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  172 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  173 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  174 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  175 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  176 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  177 33 3 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  178 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  179 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  180 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  181 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  182 48 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  183 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  184 47 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  185 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  186 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  187 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  188 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  189 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  190 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  191 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  192 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  193 43 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  194 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  195 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  196 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  197 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  198 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  199 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  200 45 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  201 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  202 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  203 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  204 53 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  205 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  206 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  207 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  208 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  209 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  210 144 21 [ 0.         1.1875     1.48       2.2909375  3.60625    4.275    ]
>>:  211 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  212 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  213 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  214 49 0 [ 0.         1.0875     1.48       2.2909375  3.6765625  4.4      ]
>>:  215 49 4 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  216 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  217 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  218 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  219 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  220 51 0 [ 0.        1.0875    1.46      2.325625  3.60625   4.475   ]
>>:  221 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  222 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  223 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  224 30 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  225 64 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  226 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  227 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  228 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  229 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  230 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  231 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  232 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  233 47 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  234 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  235 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  236 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  237 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  238 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  239 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  240 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  241 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  242 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  243 59 5 [ 0.         1.0875     1.76       2.2909375  3.278125   4.475    ]
>>:  244 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  245 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  246 56 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  247 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  248 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  249 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  250 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  251 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  252 63 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  253 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  254 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  255 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  256 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  257 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  258 48 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  259 162 31 [ 0.         1.1875     1.48       2.2909375  3.60625    4.275    ]
>>:  260 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  261 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  262 46 0 [ 0.         1.0875     1.48       2.2909375  3.6765625  4.4      ]
>>:  263 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  264 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  265 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  266 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  267 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  268 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  269 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  270 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  271 62 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  272 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  273 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  274 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  275 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  276 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  277 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  278 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  279 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  280 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  281 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  282 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  283 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  284 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  285 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  286 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  287 58 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  288 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  289 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  290 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  291 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  292 45 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  293 45 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  294 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  295 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  296 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  297 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  298 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  299 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  300 67 11 [ 0.         1.0875     1.76       2.2909375  3.278125   4.475    ]
>>:  301 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  302 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  303 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  304 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  305 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  306 65 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  307 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  308 46 0 [ 0.         1.0875     1.48       2.81125    3.2546875  4.475    ]
>>:  309 72 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  310 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  311 38 12 [ 0.         1.0875     1.82       2.2909375  3.60625    4.05     ]
>>:  312 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  313 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  314 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  315 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  316 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  317 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  318 48 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  319 43 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  320 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  321 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  322 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  323 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  324 41 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  325 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  326 55 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  327 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  328 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  329 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  330 57 82 [ 0.         0.9625     1.48       2.6378125  3.60625    4.475    ]
>>:  331 61 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  332 51 0 [ 0.        1.0875    1.46      2.325625  3.60625   4.475   ]
>>:  333 38 81 [ 0.         0.9625     1.48       2.6378125  3.60625    4.475    ]
>>:  334 48 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  335 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  336 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  337 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  338 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  339 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  340 44 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  341 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  342 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  343 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  344 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  345 61 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  346 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  347 42 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  348 45 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  349 33 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  350 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  351 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  352 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  353 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  354 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  355 42 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  356 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  357 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  358 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  359 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  360 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  361 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  362 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  363 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  364 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  365 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  366 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  367 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  368 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  369 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  370 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  371 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  372 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  373 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  374 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  375 48 0 [ 0.        1.0875    1.46      2.325625  3.60625   4.475   ]
>>:  376 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  377 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  378 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  379 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  380 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  381 35 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  382 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  383 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  384 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  385 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  386 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  387 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  388 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  389 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  390 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  391 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  392 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  393 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  394 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  395 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  396 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  397 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  398 52 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  399 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  400 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  401 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  402 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  403 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  404 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  405 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  406 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  407 40 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  408 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  409 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  410 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  411 51 77 [ 0.         0.975      1.66       2.2909375  3.60625    4.475    ]
>>:  412 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  413 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  414 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  415 55 1 [ 0.         1.0875     1.48       2.2909375  3.6765625  4.4      ]
>>:  416 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  417 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  418 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  419 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  420 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  421 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  422 47 0 [ 0.         1.0875     1.48       2.9153125  3.60625    4.025    ]
>>:  423 48 3 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  424 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  425 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  426 40 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  427 31 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  428 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  429 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  430 53 2 [ 0.         1.0875     1.82       2.2909375  3.60625    4.05     ]
>>:  431 36 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  432 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  433 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  434 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  435 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  436 46 79 [ 0.         0.975      1.66       2.2909375  3.60625    4.475    ]
>>:  437 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  438 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  439 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  440 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  441 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  442 45 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  443 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  444 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  445 33 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  446 45 0 [ 0.        1.0875    1.46      2.325625  3.60625   4.475   ]
>>:  447 49 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  448 44 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  449 51 99 [ 0.         0.975      1.66       2.2909375  3.60625    4.475    ]
>>:  450 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  451 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  452 60 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  453 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  454 70 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  455 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  456 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  457 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  458 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  459 43 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  460 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  461 48 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  462 36 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  463 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  464 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  465 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  466 176 29 [ 0.         1.1875     1.48       2.2909375  3.60625    4.275    ]
>>:  467 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  468 59 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  469 61 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  470 55 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  471 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  472 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  473 61 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  474 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  475 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  476 47 2 [ 0.         1.0875     1.48       2.9153125  3.60625    4.025    ]
>>:  477 39 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  478 51 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  479 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  480 46 8 [ 0.         1.0875     1.76       2.2909375  3.278125   4.475    ]
>>:  481 41 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  482 46 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  483 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  484 38 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  485 46 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  486 191 19 [ 0.         1.15       1.48       2.2909375  3.4890625  4.475    ]
>>:  487 37 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  488 58 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  489 56 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  490 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  491 53 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  492 43 1 [ 0.         1.0875     1.48       2.9153125  3.60625    4.025    ]
>>:  493 47 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  494 50 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  495 57 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  496 42 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  497 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  498 52 1 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]
>>:  499 54 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]

>>:  223 60 1 [ 0.    1.1   1.5   2.45  3.2   4.05]
>>:  294 50 0 [ 0.    1.05  1.34  2.44  3.35  4.06]
>>:  497 45 0 [ 0.         1.0875     1.48       2.2909375  3.60625    4.475    ]